In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
CH_vcf = pd.read_excel('gdrive/My Drive/CH-tissue/ch_added.xlsx',  header=0)
Tissue_vcf = pd.read_excel('gdrive/My Drive/CH-tissue/labels.xlsx', sheet_name='Tumor', header=0)
unlabel_vcf = pd.read_excel('gdrive/My Drive/CH-tissue/unlabel.xlsx', sheet_name=0, header=0)
unlabel_vcf1 = pd.read_excel('gdrive/My Drive/CH-tissue/unlabel4.xlsx', sheet_name=0, header=0)
unlabel_vcf2 = pd.read_excel('gdrive/My Drive/CH-tissue/unlabel5.xlsx', sheet_name=0, header=0)
unlabel_vcf3 = pd.read_excel('gdrive/My Drive/CH-tissue/unlabel6.xlsx', sheet_name=0, header=0)
unlabel_vcf4 = pd.read_excel('gdrive/My Drive/CH-tissue/unlabel7.xlsx', sheet_name=0, header=0)
CH_seq = open('gdrive/My Drive/CH-tissue/ch-seq.txt' , 'r').readlines()
tissue_seq = open('gdrive/My Drive/CH-tissue/tissue-seq.txt' , 'r') .readlines()
unlabel_seq = open('gdrive/My Drive/CH-tissue/unlabel-seq.txt' , 'r').readlines()
unlabel_seq1 = open('gdrive/My Drive/CH-tissue/unlabel4-seq.txt' , 'r').readlines()
unlabel_seq2 = open('gdrive/My Drive/CH-tissue/unlabel5-seq.txt' , 'r').readlines()
unlabel_seq3 = open('gdrive/My Drive/CH-tissue/unlabel6-seq.txt' , 'r').readlines()
unlabel_seq4 = open('gdrive/My Drive/CH-tissue/unlabel7-seq.txt' , 'r').readlines()

In [ ]:
Tissue_vcf = Tissue_vcf.rename(columns= {'CHROM':'Chromosome' , 'POSITION': 'Start_Position' , 'REF':'Ref' , 'ALT' : 'Alt'})
Tissue_vcf['label'] = 'tissue'

In [ ]:
CH_vcf = CH_vcf.rename(columns= {'chromosome':'Chromosome' , 'base_pair_location': 'Start_Position' , 'effect_allele':'Ref' , 'other_allele' : 'Alt'})

In [ ]:
glioma_vcf = pd.read_excel('gdrive/My Drive/CH-tissue/glioma.xlsx', sheet_name=0, header=0)
glioma_seq = open('gdrive/My Drive/CH-tissue/glioma.txt' , 'r').readlines()

In [ ]:
def Preprocessing(df, seq, p=3, Label=True):
    dic = {'A':0, 'T':1, 'C':2, 'G':3}
    if Label:
        df = df[['Chromosome', 'Start_Position', 'Alt' , 'Ref','label']]
    else:
        df = df[['Chromosome', 'Start_Position', 'Alt' , 'Ref']]
    for i in range(1,p+1):
        df[str(i)+' position'] = 0
        df[str(-1*i)+' position'] = 0
    Filtered = []
    df['Ref'].replace(['-'], ['None'], inplace=True)
    df['Alt'].replace(['-'], ['None'], inplace=True)
    #df = df.loc[(df['Ref'].isin(n)) & (df['Alt'].isin(n))]

    index = 0

    for i in range(len(seq)):

      if seq[i][0]=='>':
        index+=1
      elif seq[i][0]!='N':

        if len(df['Ref'][index-1])==1 and len(df['Alt'][index-1])==1:
            for j in range(1,p+1):
              df[str(j)+' position'][index-1] = dic[seq[i][10+j]]
              df[str(-1*j)+' position'][index-1] = dic[seq[i][10-j]]
            Filtered.append(index-1)

    df = df.iloc[Filtered]
    df = df.reset_index(drop=True)
    df['Ref'].replace(['A','T','C','G'], [0,1,2,3], inplace=True)
    df['Alt'].replace(['A','T','C','G'], [0,1,2,3], inplace=True)
    if Label:
        df['label'].replace(['tissue','CH'], [0,1], inplace=True)

    df.loc[df['Chromosome']=='X', ['Chromosome']] = 23
    df.loc[df['Chromosome']=='Y', ['Chromosome']] = 24
    #df['Start_Position'] = 10*(df['Start_Position'] - df['Start_Position'].mean())/df['Start_Position'].std()
    df['Start_Position'] = (df['Start_Position']-100000000)/120000000
    return df

CH = Preprocessing(CH_vcf , CH_seq)
Tissue = Preprocessing(Tissue_vcf , tissue_seq)

In [ ]:
Tissue = Tissue.sample(12500)
Tissue = Tissue.reset_index(drop=True)
Tissue.tail()

In [ ]:
glioma_vcf['chr'] = glioma_vcf['chr'].map(lambda x: 'chr'+str(x))

In [ ]:
def Pre_Process(unlabel_vcf, unlabel_seq):
    unlabeled= unlabel_vcf[['chr','pos','ref','alt']]
    dic= {}
    for i in range(22):
        dic['chr'+str(i+1)] = i+1
    dic['chrX'] = 23
    dic['chrY'] = 24
    chromosomes = list(dic.keys())

    unlabeled = unlabeled.loc[unlabeled['chr'].isin(chromosomes)]
    unlabeled['chr'].replace(chromosomes, list(dic.values()), inplace=True)

    unlabels = unlabeled.rename(columns= {'chr':'Chromosome' , 'pos': 'Start_Position' , 'ref':'Ref' , 'alt' : 'Alt'})

    unlabels = Preprocessing(unlabels , unlabel_seq , Label=False)
    return unlabels

unlabels1 = Pre_Process(unlabel_vcf, unlabel_seq)
unlabels2 = Pre_Process(unlabel_vcf1, unlabel_seq1)
unlabels3 = Pre_Process(unlabel_vcf2, unlabel_seq2)
unlabels4 = Pre_Process(unlabel_vcf3, unlabel_seq3)
unlabels5 = Pre_Process(unlabel_vcf4, unlabel_seq4)

glioma = Pre_Process(glioma_vcf, glioma_seq)

In [ ]:
unlabels = pd.concat([unlabels1, unlabels2, unlabels3, unlabels4, unlabels5])

In [ ]:
total_labels = pd.concat([CH,Tissue]).reset_index(drop = True)
test_labels = total_labels.sample(frac=0.1)
train_labels = total_labels.drop(test_labels.index)
test_labels = test_labels.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
test_labels.tail(10)

In [ ]:
y_train = train_labels.pop("label")
y_test = test_labels.pop("label")

In [ ]:
test_glioma = glioma.sample(frac=0.2)
train_glioma = glioma.drop(test_glioma.index)
test_glioma = test_glioma.reset_index(drop=True)
train_glioma = train_glioma.reset_index(drop=True)

X_train_unsup_glioma = train_glioma.to_numpy()
X_test_unsup_glioma = test_glioma.to_numpy()

X_train_unsup_glioma = np.asarray(X_train_unsup_glioma).astype('float32')
X_test_unsup_glioma = np.asarray(X_test_unsup_glioma).astype('float32')

In [ ]:
X_train = train_labels.to_numpy()
X_test = test_labels.to_numpy()

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

unlabels = pd.concat([train_labels, unlabels])
test_unlabels = unlabels.sample(frac=0.1)
train_unlabels = unlabels.drop(test_unlabels.index)
test_unlabels = test_unlabels.reset_index(drop=True)
train_unlabels = train_unlabels.reset_index(drop=True)

X_train_unsup = train_unlabels.to_numpy()
X_test_unsup = test_unlabels.to_numpy()

X_train_unsup = np.asarray(X_train_unsup).astype('float32')
X_test_unsup = np.asarray(X_test_unsup).astype('float32')

In [ ]:
from xgboost import XGBRegressor
from sklearn.linear_model import BayesianRidge, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [ ]:
def define_gen(latent_dim, output_dim):
  Input_gen = tf.keras.layers.Input(shape=(latent_dim,))
  X = tf.keras.layers.Dense(50, activation= tf.keras.layers.LeakyReLU(alpha=0.2))(Input_gen)
  X = tf.keras.layers.Dropout(0.2)(X)
  X = tf.keras.layers.Dense(40, activation= tf.keras.layers.LeakyReLU(alpha=0.2))(X)
  X = tf.keras.layers.Dropout(0.2)(X)
  X = tf.keras.layers.Dense(30, activation= tf.keras.layers.LeakyReLU(alpha=0.2))(X)
  X = tf.keras.layers.Dropout(0.2)(X)
  X = tf.keras.layers.Dense(20, activation= tf.keras.layers.LeakyReLU(alpha=0.2))(X)

  Output_gen = tf.keras.layers.Dense(output_dim)(X)
  model = tf.keras.models.Model(Input_gen, Output_gen)
  return model

In [ ]:
def define_disc(input_shape):
  Input_disc = tf.keras.layers.Input(shape= input_shape)
  X = tf.keras.layers.Conv1D(filters=16 , kernel_size=3, activation= tf.keras.layers.LeakyReLU(alpha=0.2))(Input_disc)
  X = tf.keras.layers.Conv1D(filters=32 , kernel_size=3, activation= tf.keras.layers.LeakyReLU(alpha=0.2))(X)
  X = tf.keras.layers.Conv1D(filters=64 , kernel_size=3, activation= tf.keras.layers.LeakyReLU(alpha=0.2))(X)
  X = tf.keras.layers.Conv1D(filters=128 , kernel_size=3, activation= tf.keras.layers.LeakyReLU(alpha=0.2))(X)
  Output_disc = tf.keras.layers.Flatten()(X)
  #Output_disc = tf.keras.layers.Dense(40)(X)
  model= tf.keras.models.Model(Input_disc,Output_disc)
  return model

In [ ]:
def custom_activation(x):
    Z_x = tf.keras.backend.sum(tf.keras.backend.exp(x), axis=-1, keepdims=True)
    D_x = Z_x /(Z_x+1)
    return D_x

In [ ]:
def define_supdisc(disc, num_classes=2):
  disc.trainable = False
  model=tf.keras.models.Sequential()
  model.add(disc)
  model.add(tf.keras.layers.Dense(80, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(60, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(40, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(20, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(10, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dense(8, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dense(6, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dense(4, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dense(1, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Activation('sigmoid'))
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.05, beta_1=0.5),
                loss="binary_crossentropy",metrics=['AUC',])
  return model

In [ ]:
def define_unsupdisc(disc):
  model=tf.keras.models.Sequential()
  model.add(disc)
  model.add(tf.keras.layers.Dense(80, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(60, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(40, activation= tf.keras.layers.LeakyReLU(alpha=0.2)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(20,activation='sigmoid'))
  model.add(tf.keras.layers.Dense(10,activation='sigmoid'))
  model.add(tf.keras.layers.Dense(5,activation='sigmoid'))
  model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(lr=0.05, beta_1=0.5))
  return model

In [ ]:
def define_gan(gen_model, disc_unsup):
	disc_unsup.trainable = False
	gan_output = disc_unsup(gen_model.output)
	model = tf.keras.models.Model(gen_model.input, gan_output)
	model.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(lr=0.01, beta_1=0.5))
	return model

In [ ]:
def generate_latent_points(latent_dim, n_samples):
	z_input = np.random.randn(latent_dim * n_samples)
	z_input = z_input.reshape(n_samples, latent_dim)
	return z_input

In [ ]:
def generate_fake_samples(generator, latent_dim, n_samples):
	z_input = generate_latent_points(latent_dim, n_samples)
	fake_images = generator.predict(z_input)
	y = np.zeros((n_samples, 1))
	return fake_images, y

In [ ]:
def generate_unlabeled_samples(X_unsup, n_samples, batch_size):
  ix = np.random.randint(0, X_unsup.shape[0], n_samples*batch_size)
  X = X_unsup[ix]
  return X

In [ ]:
def generate_labeled_samples(X_sup, y_sup, n_samples, batch_size):
  ix = np.random.randint(0, X_sup.shape[0], n_samples*batch_size)
  X = X_sup[ix]
  y = y_sup[ix]
  return [X, y]

In [ ]:
def train(disc_unsup, disc_sup, gan_model, gen_model, X_train_unsup, X_test_unsup, X_train, y_train, X_test, y_test, latent_dim, num_steps, batch_size):
  history_sup=[]
  history_sup_train=[]

  n_samples = (X_train_unsup.shape[0]//batch_size)

  #Unsupervised training
  for i in range(num_steps):

      X_fake, y_fake = generate_fake_samples(gen_model, latent_dim, X_test_unsup.shape[0])
      y_real =  np.ones(shape=(X_test_unsup.shape[0],1))
      d_loss_real = disc_unsup.test_on_batch(X_test_unsup.reshape(X_test_unsup.shape[0],-1,1), y_real)
      d_loss_fake = disc_unsup.test_on_batch(X_fake.reshape(X_test_unsup.shape[0],-1,1), y_fake)
      X_gan, y_gan = generate_latent_points(latent_dim, X_test_unsup.shape[0]),np.ones((X_test_unsup.shape[0], 1))
      gan_loss = gan_model.test_on_batch(X_gan, y_gan)

      print('test-unsupervise>%d, d[%.3f,%.3f], g[%.3f]' % (i+1, d_loss_real, d_loss_fake, gan_loss))

      X_fake, y_fake = generate_fake_samples(gen_model, latent_dim, X_trian_unsup.shape[0])
      y_real =  np.ones(shape=(X_train_unsup.shape[0],1))
      d_loss_real = disc_unsup.test_on_batch(X_train_unsup.reshape(X_train_unsup.shape[0],-1,1), y_real)
      d_loss_fake = disc_unsup.test_on_batch(X_fake.reshape(X_train_unsup.shape[0],-1,1), y_fake)
      X_gan, y_gan = generate_latent_points(latent_dim, X_train_unsup.shape[0]),np.ones((X_train_unsup.shape[0], 1))
      gan_loss = gan_model.test_on_batch(X_gan, y_gan)

      print('train-unsupervise>%d, d[%.3f,%.3f], g[%.3f]' % (i+1, d_loss_real, d_loss_fake, gan_loss))

      X_real = generate_unlabeled_samples(X_train_unsup, n_samples, batch_size)
      for j in range(n_samples):
        X_fake, y_fake = generate_fake_samples(gen_model, latent_dim, batch_size)
        y_real =  np.ones(shape=(batch_size,1))
        X_Real = X_real[batch_size*j:batch_size*(j+1)].reshape(batch_size,-1,1)
        X_Fake = X_fake.reshape(batch_size,-1,1)
        d_loss_real = disc_unsup.train_on_batch(X_Real, y_real)
        d_loss_fake = disc_unsup.train_on_batch(X_Fake, y_fake)
        X_gan, y_gan = generate_latent_points(latent_dim, batch_size),np.ones((batch_size, 1))
        gan_loss = gan_model.train_on_batch(X_gan, y_gan)

  # Suoervised traning
  for i in range(num_steps):

      sup_loss, sup_acc = disc_sup.test_on_batch(X_test.reshape(X_test.shape[0],-1,1), y_test)
      history_sup.append(sup_acc)

      print('test-supervise>%d, c[%.3f,%.0f]' % (i+1, sup_loss, sup_acc*100))

      sup_loss, sup_acc = disc_sup.test_on_batch(X_train.reshape(X_train.shape[0],-1,1), y_train)
      history_sup_train.append(sup_acc)

      print('train-supervise>%d, c[%.3f,%.0f]' % (i+1, sup_loss, sup_acc*100))

      X , y = generate_labeled_samples(X_train, y_train, n_samples, batch_size)
      for j in range(n_samples):
        X_sup = X[batch_size*j:batch_size*(j+1)].reshape(batch_size,-1,1)
        sup_loss, sup_acc = disc_sup.train_on_batch(X_sup, y[batch_size*j:batch_size*(j+1)])

  return [history_sup, history_sup_train]

In [ ]:
num_steps = 200
latent_dim = 50
batch_size = 2000
disc=define_disc((X_train[0].shape[0],1))
disc_sup=define_supdisc(disc)
disc_unsup=define_unsupdisc(disc)

gen_model = define_gen(latent_dim,10)
gan_model = define_gan(gen_model, disc_unsup)

In [ ]:
[history_sup ,history_sup_train] = train(disc_unsup, disc_sup, gan_model, gen_model, X_train_unsup, X_test_unsup, X_train, y_train, X_test, y_test, latent_dim, 200, batch_size)

In [ ]:
[history_sup2  ,history_sup_train2] = train(disc_unsup, disc_sup, gan_model, gen_model, X_train_unsup_glioma, X_test_unsup_glioma, X_train, y_train, X_test, y_test, latent_dim, 200, batch_size)

In [ ]:
font = {
        'size'   : 15,
        'weight' : 'bold'}
plt.rcParams["font.family"] = "Helvetica"
plt.rc('font', **font)
plt.rc('text', usetex=False)

In [ ]:
x1 = [i*10 for i in range(1,num_steps+1)]
x2 = [i*10 for i in range(1,num_steps+1)]

hfont = {'fontname':'Helvetica'}

plt.plot(x1, history_sup , c = 'r')
plt.plot(x2, history_sup_train , c = '#000000')
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['test', 'train'])

plt.savefig('gdrive/My Drive/modelacc.png')
plt.show()

In [ ]:
def ROCcurve(y_label,y_pred):
  n=500
  x=[]
  y=[]
  theta=[i/n for i in range(n+1)]
  TrueTP = y_label.sum()
  TrueFP = len(y_label) - TrueTP
  for p in theta:
      TP=0
      FP=0
      for j in range(len(y_pred)):
        if y_pred[j]>=p:
          if y_label[j]==1:
            TP+=1
          else:
            FP+=1
      x.append(FP/TrueFP)
      y.append(TP/TrueTP)
  return [x,y]

In [ ]:
y_pred = disc_sup.predict(X_test)
y_pred2 = disc_sup.predict(X_train)

[x1,y1]=ROCcurve(y_test,y_pred)
[x2,y2]=ROCcurve(y_train,y_pred2)

701/701 [==============================] - 1s 1ms/step


In [ ]:
plt.plot(x1, y1 , c = 'r', linewidth = '2.5')
plt.plot(x2, y2 , c = '#000000', linewidth = '2.5')

plt.title("ROC Curve", fontsize=20)
plt.xlabel("False Positive Rate", fontsize=18)
plt.ylabel("True Positive Rate", fontsize=18)

plt.legend(['test', 'train'])

plt.savefig('gdrive/My Drive/ROCglioma.png')
plt.show()

In [ ]:
model_factory = [
 MLPRegressor(),
 BayesianRidge()
]

colores = ['#000099' ,'#000000' , '#990000' ]

from sklearn import metrics
I = 0
for model in model_factory:
  clf = model
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  y_pred[y_pred<0]=0
  [x,y] = ROCcurve(y_test, y_pred )
  plt.plot(x,y, c = colores[I])
  I+=1

y_pred = disc_sup.predict(X_test)
[x,y]=ROCcurve(y_test,y_pred )
plt.plot(x,y , colores[2])
plt.legend([
 'MLPRegressor',
 'BayesianRidge',
 'SSGAN'
])
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.savefig('gdrive/My Drive/ROC.png')
plt.show()

In [ ]:
def Acc(y_label, y_pred):
  C = 0
  for Indx in range(len(y_pred)):
    if y_pred[Indx]>=0.5 and y_label[Indx]==1:
      C+=1
    elif y_pred[Indx]<=0.5 and y_label[Indx]==0:
      C+=1
  return C/len(y_pred)

In [ ]:
P = [i*0.05 for i in range(7,21)]
y1 = []
y2 = []
y3 = []
y4 = []

Acc_models = {}
Acc_models['p'] = P
Acc_models['MLPRegressor'] = []
Acc_models['BayesianRidge'] = []
Acc_models['Kneighborsregressor'] = []
Acc_models['SSGAN'] = []

for p in P:
  Arr= np.random.uniform(low=0, high=1, size= len(y_train))
  X_Train = X_train[Arr<=p,:]
  y_Train = y_train[Arr<=p]
  y_Train= y_Train.reset_index()
  y_Train = y_Train['label']

  clf = KNeighborsRegressor()
  clf.fit(X_Train,y_Train)
  y_pred = clf.predict(X_test)
  Acc_models['Kneighborsregressor'].append(Acc(y_test,y_pred))

  clf = MLPRegressor()
  clf.fit(X_Train,y_Train)
  y_pred = clf.predict(X_test)
  Acc_models['MLPRegressor'].append(Acc(y_test,y_pred))

  clf = BayesianRidge()
  clf.fit(X_Train,y_Train)
  y_pred = clf.predict(X_test)
  Acc_models['BayesianRidge'].append(Acc(y_test,y_pred))

  disc=define_disc((X_train[0].shape[0],1))
  disc_sup=define_supdisc(disc)
  disc_unsup=define_unsupdisc(disc)

  gen_model = define_gen(latent_dim,10)
  gan_model = define_gan(gen_model, disc_unsup)
  train(disc_unsup, disc_sup, gan_model, gen_model, X_train_unsup, X_test_unsup, X_Train, y_Train, X_test, y_test, latent_dim, 100, 1000)
  Acc_models['SSGAN'].append(history_sup[-1])


colores= ['#000000' , '#500050', '#990000', '#000099']

plt.figure(figsize=(10,7))
plt.plot(Acc_models['p'], Acc_models['MLPRegressor'], marker = 'o' , c=colores[0])
plt.plot(Acc_models['p'], Acc_models['BayesianRidge'], marker = 'o' , c=colores[1])
plt.plot(Acc_models['p'], Acc_models['Kneighborsregressor'], marker = 'o' ,  c=colores[2])
plt.plot(Acc_models['p'], Acc_models['SSGAN'], marker = 'o' ,  c=colores[3])
plt.xlabel("%training")
plt.ylabel("Accuracy")
plt.legend(['MLPRegressor', 'BayesianRidge','Kneighborsregressor','SSGAN'])

plt.savefig('gdrive/My Drive/project2/ModelsAcc.png')
plt.show()